## Create spark context

In [1]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "2").appName("Analysis").master("local[2]").getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

/home/sunbeam/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
%run "../includes/configuration"

## Read file

In [3]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results").withColumnRenamed("position", "result_position").withColumnRenamed("grid", "result_grid").withColumnRenamed("fastest_lap", "result_fastest_lap")
results_df = spark.read.parquet(f"{processed_folder_path}/results")

In [7]:
circuit_by_speed_df = race_results_df.join(results_df, race_results_df.result_grid == results_df.grid)

## Filter year 2019 in dataframe

In [11]:
circuit_by_speed_df = circuit_by_speed_df.filter("race_year == 2019")

In [14]:
circuit_by_speed_df.createOrReplaceTempView("circuit_by_speed")

## Average speed on different circuits in year 2019

In [18]:
query = """
SELECT r.race_name, ROUND(AVG(rs.fastest_lap_speed), 2) AS avg_speed
FROM circuit_by_speed r
JOIN circuit_by_speed rs ON r.race_id = rs.race_id
GROUP BY r.race_name
ORDER BY avg_speed
"""

In [19]:
final_df = spark.sql(query)

In [21]:
final_df.show()

+--------------------+---------+
|           race_name|avg_speed|
+--------------------+---------+
|Azerbaijan Grand ...|   202.98|
|Abu Dhabi Grand Prix|   203.01|
| Japanese Grand Prix|   203.01|
|United States Gra...|   203.01|
| Austrian Grand Prix|   203.01|
|Singapore Grand Prix|   203.01|
|  Italian Grand Prix|   203.01|
|   German Grand Prix|   203.01|
|  Russian Grand Prix|   203.01|
|   French Grand Prix|   203.01|
| Canadian Grand Prix|   203.01|
|   Monaco Grand Prix|   203.01|
|  Chinese Grand Prix|   203.01|
|  Bahrain Grand Prix|   203.01|
|Brazilian Grand Prix|   203.01|
|  Mexican Grand Prix|   203.01|
|  Belgian Grand Prix|   203.01|
|Hungarian Grand Prix|   203.01|
|  British Grand Prix|   203.01|
|  Spanish Grand Prix|   203.01|
+--------------------+---------+
only showing top 20 rows



In [ ]:
query = """
SELECT r.race_name, ROUND(AVG(rs.fastest_lap_speed), 2) AS avg_speed
FROM circuit_by_speed r
JOIN circuit_by_speed rs ON r.race_id = rs.race_id
GROUP BY r.race_name
ORDER BY avg_speed
"""